In [1]:
import kagglehub
import os

dataset_path = kagglehub.dataset_download(
    'yasserhessein/tuberculosis-chest-x-rays-images'
)

print("Dataset downloaded at:", dataset_path)


100%|██████████| 26.3M/26.3M [00:02<00:00, 11.4MB/s]

Extracting files...


Dataset downloaded at: /root/.cache/kagglehub/datasets/yasserhessein/tuberculosis-chest-x-rays-images/versions/2


In [2]:
import os

INNER_PATH = os.path.join(
    dataset_path,
    "Dataset of Tuberculosis Chest X-rays Images"
)

print("Using dataset path:", INNER_PATH)
print("Subfolders:", os.listdir(INNER_PATH))


Using dataset path: /root/.cache/kagglehub/datasets/yasserhessein/tuberculosis-chest-x-rays-images/versions/2/Dataset of Tuberculosis Chest X-rays Images
Subfolders: ['Normal Chest X-rays', 'TB Chest X-rays']


In [3]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model

IMG_SIZE = (224, 224)
BATCH_SIZE = 16

datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2
)

train_data = datagen.flow_from_directory(
    INNER_PATH,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="binary",
    subset="training"
)

val_data = datagen.flow_from_directory(
    INNER_PATH,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="binary",
    subset="validation"
)

base_model = ResNet50(
    weights="imagenet",
    include_top=False,
    input_shape=(224,224,3)
)

base_model.trainable = False

x = GlobalAveragePooling2D()(base_model.output)
x = Dense(128, activation="relu")(x)
output = Dense(1, activation="sigmoid")(x)

model = Model(base_model.input, output)

model.compile(
    optimizer="adam",
    loss="binary_crossentropy",
    metrics=["accuracy"]
)

model.fit(train_data, validation_data=val_data, epochs=5)

model.save("tb_model.h5")


Found 2408 images belonging to 2 classes.
Found 600 images belonging to 2 classes.
94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 5s 0us/step
Epoch 1/5


/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


151/151 ━━━━━━━━━━━━━━━━━━━━ 575s 4s/step - accuracy: 0.8359 - loss: 0.4681 - val_accuracy: 0.8300 - val_loss: 0.4423
Epoch 2/5
151/151 ━━━━━━━━━━━━━━━━━━━━ 567s 4s/step - accuracy: 0.8377 - loss: 0.4490 - val_accuracy: 0.8300 - val_loss: 0.4448
Epoch 3/5
151/151 ━━━━━━━━━━━━━━━━━━━━ 568s 4s/step - accuracy: 0.8361 - loss: 0.4209 - val_accuracy: 0.8300 - val_loss: 0.4299
Epoch 4/5
151/151 ━━━━━━━━━━━━━━━━━━━━ 565s 4s/step - accuracy: 0.8327 - loss: 0.4183 - val_accuracy: 0.8300 - val_loss: 0.4070
Epoch 5/5
151/151 ━━━━━━━━━━━━━━━━━━━━ 567s 4s/step - accuracy: 0.8365 - loss: 0.3890 - val_accuracy: 0.8300 - val_loss: 0.3745


In [4]:
import os
print(os.listdir("."))



['.config', 'tb_model.h5', 'sample_data']


In [5]:
import os
os.getcwd()


'/content'